## Interpolate low confidence tracking points

It can be useful to interpolate over points where the confidence in the prediction is low -- this interpolation gives smoother trajectories than the noisy, low-confidence estimates. 

In [1]:
from glob import glob 
import os 
from behaveml import VideosetDataFrame, clone_metadata
from behaveml import interpolate_lowconf_points
import deeplabcut
from tqdm import tqdm

/home/blansdel/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/deeplabcut/__init__.py:78: UserWarning: 
        As PyTorch is not installed, unsupervised identity learning will not be available.
        Please run `pip install torch`, or ignore this warning.
        
  warnings.warn(


Set paths and parameters

In [3]:

#Path to DLC project file
path_config_file = '/home/blansdel/projects/behavior/dlc_projects/overnight_videos/Overnight_Videos_813a_816b_combined_iter0/config.yaml'

#Path to videos. These videos should have already been processed by DLC 
base_video_path = '/home/blansdel/ImageAnalysisScratch/AnimalBehaviorCore/Zakharenko/animal-behavior-ml/inference/Overnight\ Videos/'
#base_video_path = '/home/blansdel/ImageAnalysisScratch/AnimalBehaviorCore/Zakharenko/animal-behavior-ml/inference/Overnight\ Videos\ LSD/'

cameras_to_process = ['e3v813a', 'e3v816b'] #Only process videos from these cameras
video_pattern = '**/**/*cropped.mp4'

create_video = True #If true, will make new labeled videos with interpolated data

shuffle = 1 #DLC model shuffle. Often 1
track_method = 'ellipse' #DLC model track method. Often 'ellipse'
fps = 30 #DLC model fps


Check these to see it found the videos you want to process

In [4]:
videos = sorted(glob(os.path.join(base_video_path, video_pattern)))
these_videos = [v for v in videos if v.split('/')[-1].split('-')[0] in cameras_to_process]
dirs = list(set([os.path.dirname(v) for v in these_videos]))

Process the files

In [ ]:
#Do this by directory
for video_path in tqdm(dirs):
    output_video_path = os.path.join(video_path, 'interpolated')
    os.makedirs(output_video_path, exist_ok = True)
    tracking_files = sorted(glob(os.path.join(video_path, '*_filtered.csv')))
    this_dir_videos = sorted(glob(os.path.join(video_path, '*cropped.mp4')))
    this_dir_pickles = sorted(glob(os.path.join(video_path, '*.pickle')))
    metadata = clone_metadata(tracking_files, fps = fps)

    dataset = VideosetDataFrame(metadata)
    interpolate_lowconf_points(dataset)
    dataset.to_dlc_csv(output_video_path, save_h5_too = True)

    #link videos to output_video_path
    for pick in this_dir_pickles:
        if not os.path.exists(os.path.join(output_video_path, os.path.basename(pick))):
            os.symlink(pick, os.path.join(output_video_path, os.path.basename(pick)))

    for video in this_dir_videos:
        if not os.path.exists(os.path.join(output_video_path, os.path.basename(video))):
            os.symlink(video, os.path.join(output_video_path, os.path.basename(video)))
    sym_link_videos = sorted(glob(os.path.join(output_video_path, '*cropped.mp4')))

    if create_video:
        deeplabcut.create_labeled_video(path_config_file, sym_link_videos, videotype='.mp4', 
                                    filtered=True, shuffle = shuffle, trainingsetindex = 0, 
                                    track_method = track_method, fastmode = True,
                                    color_by = 'individual', draw_skeleton = True)